In [3]:
import numpy as np
from netCDF4 import Dataset
import datetime
from scipy.interpolate import griddata
import tqdm
from regrid import regrid
import matplotlib.pyplot as plt
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

In [ ]:
datapath = '/Users/carmennab/Dropbox/OI_PolarSnow/data/CDS_icetype/'

days = np.arange(0,4138) # until 30/04/2021
dates = [datetime.date(2010,1,1)+datetime.timedelta(days=int(days)) for days in days]
ease_lats = np.load('/Users/carmennab/Dropbox/alpha_retracker/data/auxiliary/lat_25km_cent.npy')
ease_lons = np.load('/Users/carmennab/Dropbox/alpha_retracker/data/auxiliary/lon_25km_cent.npy')

icetype_grid_25 = np.full((len(days),361,361),np.nan)

for day in tqdm.tqdm(days):
    date = dates[day].strftime('%Y%m%d')
    try:
        icetype = Dataset(datapath + 'ice_type_nh_ease2-250_cdr-v2p0_' + date + '1200.nc')
        icetype_lon = np.array(icetype['lon'])
        icetype_lat = np.array(icetype['lat'])
        itype = np.array(icetype['ice_type']).astype('float32')[0,:,:]
        itype[itype>3] = np.nan
        itype[itype<2] = np.nan
        icetype_grid_25[day,:,:] = regrid(itype,icetype_lon,icetype_lat,ease_lons,ease_lats)
    except:
        pass

In [8]:
# create icetype NETCDF
data_vars = {'Ice Type':(['t','x','y'],icetype_grid_25)}
ds = xr.Dataset(data_vars = data_vars,
                coords={'Longitude':(['x','y'],ease_lons),
                        'Latitude':(['x','y'],ease_lats),
                         'Day':(['t'],days)}     )
ds.to_netcdf(f'/Users/carmennab/Dropbox/alpha_retracker/data/auxiliary/icetype_01012010-30042021.nc','w')